In [1]:
# Load dataset

import json
input_file = "dataset/org_with_cefr_labels/us_RL.jsonl"
dataset = [json.loads(line.strip()) for line in open(input_file)]
dataset = dataset[:2]


In [ ]:
# Augment dataset with rollouts

from vllm import SamplingParams
from inference import TextGenerator

model_name = "google/gemma-3-1b-it"
sampling_params_list = [
    SamplingParams(temperature=0.3, top_p=0.85, n=2, max_tokens=1024),
    SamplingParams(temperature=0.7, top_p=0.9, n=2, max_tokens=1024),
]

text_generator = TextGenerator(model_name, sampling_params_list)

text_generator.generate_dataset(dataset)




In [4]:
# Augment dataset with rewards

classifier = pipeline(
    "text-classification", 
    model="AbdullahBarayan/ModernBERT-base-doc_en-Cefr",
    device=0,
    top_k=None,
    attn_implementation="flash_attention_2", torch_dtype=torch.float16
)

# import json
# from tqdm import tqdm
# from bert_score import score

# cefr_labels = ["A1", "A2", "B1", "B2", "C1", "C2"]
# prediction_file = "dataset/predictions/random.jsonl"

# cefr_scores = {label: [] for label in cefr_labels}
# bert_scores = {label: [] for label in cefr_labels}

# dataset = [line.strip() for line in open(prediction_file)]

# for i in tqdm(range(len(dataset))):
    
#     pred_json = json.loads(dataset[i])

#     predictions = pred_json["predictions"]
#     texts = [predictions[label] for label in cefr_labels]
    
#     cefr_probs = classifier(texts, batch_size=6)

#     candidates = []
#     references = [pred_json["summary"]] * 6
#     for label, single_probs in zip(cefr_labels, cefr_probs):
#         single_probs = {p["label"]: p["score"] for p in single_probs}
#         cefr_scores[label].append(single_probs[label])
#         candidates.append(predictions[label])

#     _, _, f1 = score(candidates, references, model_type="microsoft/deberta-xlarge-mnli", lang="en", verbose=False)
#     single_scores = f1.data.tolist()

#     for label, bscore in zip(cefr_labels, single_scores):
#         bert_scores[label].append(bscore)


Processed prompts: 100%|██████████| 24/24 [00:13<00:00,  1.84it/s, est. speed input: 3241.88 toks/s, output: 1881.37 toks/s]


In [ ]:
# Construct DPO dataset
